In [1]:
import numpy as np 
import pandas as pd
import networkx as nx
import matplotlib as plt 
import markov_clustering as mc
import random
from itertools import combinations

In [2]:
run rome.py

In [130]:
data = pd.read_csv("rome_tension.csv")

In [31]:
data = data[['rome5','BE_id','T']]
data = data.pivot(index='BE_id',columns='rome5',values='T')

In [5]:
romes = list(graph.keys())

### Undirected graph

In [125]:
G = nx.Graph()
G.add_nodes_from(romes)
weights = {}
R = 1
for rome1 in romes:
    weights[rome1] = {}
    s = 0
    for rome2 in graph[rome1]:
        if True: # replace by rome1[0] = rome2[0]
            weights[rome1][rome2] = np.float_power(data[rome1].corr(data[rome2]),R)
            s += weights[rome1][rome2]
    for rome2 in graph[rome1]:
        if True: # replace by rome1[0] = rome2[0]
             weights[rome1][rome2] =  weights[rome1][rome2]/s

for rome1 in romes:
    current_edges = list(G.edges)
    for rome2 in graph[rome1]:
        if True: # replace by rome1[0] = rome2[0]
            if rome1 not in graph[rome2]: w = weights[rome1][rome2]/2
            if rome1 in graph[rome2]: w = (weights[rome1][rome2]+weights[rome2][rome1])
            if (rome2,rome1) not in current_edges:
                G.add_edge(rome1,rome2,weight=w)
                
matrix = nx.adjacency_matrix(G)
result = mc.run_mcl(matrix,inflation=2.0)
clusters = mc.get_clusters(result)
len(clusters)

225

In [112]:
cl = [[romes[x] for x in c] for c in clusters]

### Directed graph

In [126]:
DG = nx.DiGraph()
DG.add_nodes_from(romes)
weights = {}
R = 1
for rome1 in romes:
    weights[rome1] = {}
    s = 0
    for rome2 in graph[rome1]:
        if True: # replace by rome1[0] = rome2[0]
            weights[rome1][rome2] = np.float_power(data[rome1].corr(data[rome2]),R)
            s += weights[rome1][rome2]
    for rome2 in graph[rome1]:
        w = weights[rome1][rome2]/s
        if w > 0.2: # replace by rome1[0] = rome2[0]
            DG.add_edge(rome1,rome2)

matrix = nx.adjacency_matrix(DG)
result = mc.run_mcl(matrix,inflation=2)
clusters = mc.get_clusters(result)
len(clusters) 

177

In [129]:
cl = {f'g{i}':[romes[x] for x in c] for i,c in enumerate(clusters)}
cl

{'g0': ['E1106', 'E1108', 'E1202', 'E1201', 'E1203'],
 'g1': ['E1107', 'E1103'],
 'g2': ['E1104',
  'H1501',
  'M1806',
  'M1805',
  'M1804',
  'M1803',
  'H1102',
  'H1101',
  'H1206'],
 'g3': ['E1105', 'E1103', 'E1305', 'L1303'],
 'g4': ['E1102',
  'L1401',
  'L1103',
  'L1101',
  'L1201',
  'L1202',
  'L1203',
  'L1204',
  'G1201',
  'G1204',
  'L1301',
  'L1304'],
 'g5': ['E1101', 'K1601', 'G1202'],
 'g6': ['I1102', 'I1101', 'F1108', 'F1201', 'F1106'],
 'g7': ['I1102', 'I1101', 'F1201', 'F1106', 'F1101'],
 'g8': ['I1103', 'I1605', 'I1604', 'I1607', 'I1603', 'D1404'],
 'g9': ['N4104', 'N2203', 'G1702', 'N1101'],
 'g10': ['N4102', 'N4103', 'N4101', 'N4105', 'N1101', 'N1103', 'N1102'],
 'g11': ['H1503', 'F1602', 'J1302', 'I1306', 'I1308'],
 'g12': ['H1503', 'J1302', 'H1210', 'H1201'],
 'g13': ['H1502', 'D1405', 'J1402'],
 'g14': ['H1506', 'F1103', 'F1204', 'F1202', 'H1301', 'H1303'],
 'g15': ['H1505'],
 'g16': ['H1504',
  'F1605',
  'N2204',
  'I1602',
  'M1807',
  'H1202',
  'H1208',